In [187]:

from unicodedata import category
import numpy as np
import pandas as pd
import ipywidgets
from ydata_profiling import ProfileReport
import seaborn as sns
import xgboost as xgb
from category_encoders import TargetEncoder
from sklearn import datasets, linear_model, metrics, model_selection, svm
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

Data overview

In [188]:
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')


In [189]:
df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True, axis=1)
df_test.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True, axis=1)
df = df[df["Fare"] < 500]
#df_test = df_test[df_test["Fare"] < 500]
#profile = ProfileReport(df, title="My Report", explorative=True)
#profile.to_file("report.html")
X = df.iloc[:, 1:]
X_test_df = df_test
y = df.iloc[:, 0]
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0, random_state=42)
X_train = X
X_test = X_test_df
y_train = y


Pipeline

In [190]:
cat_pipeline = Pipeline([('imputer', SimpleImputer(strategy='most_frequent')), ('encoder', OneHotEncoder(handle_unknown='ignore'))])
num_pipeline = Pipeline([('imputer', SimpleImputer(strategy='mean')), ('scaler', StandardScaler())])
preprocessor = ColumnTransformer([('cat', cat_pipeline , ['Sex', 'Embarked']), ('num', num_pipeline, ['Fare', 'Age'])])

#pipeline = Pipeline([('preprocessor', preprocessor), ('classifier', linear_model.LogisticRegression())])
pipeline = Pipeline([('preprocessor', preprocessor), ('classifier', svm.SVC(kernel='linear'))])
pipeline.fit(X_train, y_train)
print(len(pipeline.named_steps['classifier'].support_))

392


In [191]:
df_submission = pd.read_csv('gender_submission.csv', index_col=False)
df_submission['Survived'] = pipeline.predict(X_test_df)
df_submission.columns = ['PassengerId', 'Survived']
df_submission.to_csv('submission.csv', index=False)